In [ ]:
from strands import tool
from strands import Agent

import uuid
import boto3
import os
from dotenv import load_dotenv

## 1. Model
load_dotenv()
from strands.models import BedrockModel
MODEL_ID = os.environ.get('MODEL_ID')

bedrock_model = BedrockModel(model_id=MODEL_ID)

## 2. System Prompt / Instructions
system_prompt = f"You are a helpful assistant that provides concise answers. "

## 3. Conversation Manager
from strands.agent.conversation_manager import SlidingWindowConversationManager
conv_manager = SlidingWindowConversationManager(window_size=10)

#### Bedrock Agent as Tool

In [ ]:
@tool
def agenticmemory_intelligence(query):
    """
    Answer queries related to Agentic memory and AI Agents
    
    Args:
        query: User query on the topic of interest
        
    Returns:
        summary of the content
    """
    print("CALLING KB")
    load_dotenv()

    region = os.environ.get("AWS_REGION_NAME")
    agent_id = os.environ.get("AGENT_ID")
    alias_id = os.environ.get("AGENT_ALIAS")

    bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name = region)
    session_id = str(uuid.uuid1())
    end_session = False
    enable_trace = True

    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session,
    )
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                agent_answer = data.decode('utf8')
                end_event_received = True
                return agent_answer
                # End event indicates that the request finished successfully
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

#### Langgraph Agent as Tool

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def general_intelligence(query):
    """
    Answer generic queries by searching the internet
    
    Args:
        query: User query on the topic of interest
        
    Returns:
        summary of the content
    """
    print("CALLING WEBSEARCH")

    load_dotenv()
    tavily_api_key = os.environ.get('TAVILY_API_KEY')
    model_id = os.environ.get('MODEL_ID')

    ## Model - Agent Brain
    from langchain_aws import ChatBedrock
    llm = ChatBedrock(model=model_id)

    ## Graph State
    from typing import TypedDict

    class State(TypedDict):
        query: str
        web_search: str
        final_answer: str

    def search_web(state: State):
        print("SEARCHING WEB")
        search_tool = TavilySearchResults(max_results=2)
        web_results = search_tool.invoke(state["query"])
        return {"web_search": web_results}

    def aggregator(state: State):
        print("AGGREGATING RESPONSE")
        prompt = f""" Your job is to summarize from the context provided to you.  the context includes information from web search: {state["web_search"]}"""
        final_answer = llm.invoke(prompt)           
        return {"final_answer": final_answer.content}  

    parallel_builder = StateGraph(State)

    # Add nodes
    parallel_builder.add_node("call_websearch", search_web)
    parallel_builder.add_node("aggregator", aggregator)

    parallel_builder.add_edge(START, "call_websearch")
    parallel_builder.add_edge("call_websearch", "aggregator")
    parallel_builder.add_edge("aggregator", END)

    agent = parallel_builder.compile()

    state = agent.invoke({"query": query})
    return (state["final_answer"])

In [ ]:
tool_list = [general_intelligence, agenticmemory_intelligence]

agent = Agent(model=bedrock_model,
              tools = tool_list,
              system_prompt=system_prompt, 
              conversation_manager=conv_manager
            )

In [ ]:
response = agent("who is usain bolt")

In [ ]:
response = agent("what are different types of agentic ai memory")